In [1]:
import scipy.io
import numpy as np
import decimal
import pickle
from matplotlib import pyplot as plt
from matplotlib import rcParams

In [2]:
rcParams['figure.figsize'] = (20, 12)
rcParams.update({'font.size': 16})

Функция, которая считает Q(T)

In [3]:
def do_Qt16_O3(T):
    mol = 3
    Q_arr = np.zeros(5)
    for iso in range(1, 6):
        mol = str(mol)
        iso = str(iso)
        file = 'QTpy/'+mol+'_'+iso+'.QTpy'

        QTdict = {}
        with open(file, 'rb') as handle:
            QTdict = pickle.loads(handle.read())
        if(T==int(T)):
            key=str(int(T))
            Q = QTdict[key]
        else:
            key=str(int(T))
            Q1 = float(QTdict[key])
            key=str(int(T+1))
            Q2 = float(QTdict[key])
            QT = Q1+(Q2-Q1)*(T-int(T))
            Q = QT
        Q_arr[int(iso) - 1] = Q
    return Q_arr

Функция, которая считает контур Фойгта вокруг точки w0

In [4]:
def contur_Humlicek(w, w0, wL0, wD):
    y = wL0/wD
    x = np.zeros(len(w))
    for i in range(len(w)):
        x[i] = (w[i]-w0)/wD
    t = np.zeros(len(w), dtype=np.complex_)
    for i in range(len(w)):
        t[i] = y - 1j*x[i]
    u = np.zeros(len(x), dtype=np.complex_)
    for i in range(len(x)):
        u[i] = t[i]*t[i]
    s = np.abs(x) + y    
    p = (1e-150)*np.ones(len(w))
    
    ff1 = np.where(s >= 15) # region 1 (s>=15)
    if len(ff1) != 0:
        w4 = t[ff1]*0.5641896/(0.5 + u[ff1])
        p[ff1] = np.real(w4)/(np.sqrt(np.pi)*wD)
    
    ff2 = np.where((s<15)&(s>=5.5)) # region 2 (5.5<=s<15)
    if len(ff2) != 0:
        w4 = t[ff2]*(1.410474+u[ff2]*0.5641896)/(0.75+u[ff2]*(3.0+u[ff2]))
        p[ff2] = np.real(w4)/(np.sqrt(np.pi)*wD)
    
    ff3 = np.where((s<5.5) & (y>=0.195*np.abs(x)-0.176)) # region 3 (s<5.5, y>=0.195*abs(x)-0.176)
    if len(ff3) != 0:
        w4 = (16.4955 + t[ff3]*20.20933 + u[ff3]*11.96482 + t[ff3]*u[ff3]*3.778987 + 
              u[ff3]*u[ff3]*0.5642236)/(16.4955 + t[ff3]*38.82363 + 
                                                              u[ff3]*39.27121 + t[ff3]*u[ff3]*21.69274 + 
                                                              u[ff3]*u[ff3]*6.699398 + t[ff3]*u[ff3]*u[ff3])
        p[ff3] = np.real(w4)/(np.sqrt(np.pi)*wD)
    
    ff4 = np.where((s<5.5) & (y<0.195*np.abs(x)-0.176)) # region 4 (s<5.5, y<0.195*abs(x)-0.176)
    if len(ff4) != 0:
        w4 = np.exp(u[ff4]) - t[ff4]*(36183.31 - u[ff4]*(3321.9905 - u[ff4]*(1540.787 - 
                                                                             u[ff4]*(219.0313 - u[ff4]*(35.76683 - 
                                                                                                        u[ff4]*(1.320522 - u[ff4]*0.56419))))))/(32066.6 - u[ff4]*(24322.84 - 
                                                                                                                                        u[ff4]*(9022.228 - u[ff4]*(2186.181 - 
                                                                                                                                        u[ff4]*(364.2191 - u[ff4]*(61.57037 - 
                                                                                                                                        u[ff4]*(1.841439 - u[ff4])))))))
        p[ff4]= np.real(w4)/(np.sqrt(np.pi)*wD)
    return p

Функция, считывающая параметры молекулы

In [5]:
# считывание параметров молекулы
def read_molpar16_O3(iso):
    if iso==1:
        isoN=666
        Sn=0.992901
        Q296=3483.71
        gj=1
        molM=47.984745
    elif iso==2:
        isoN=668
        Sn=0.003982
        Q296=7465.68
        gj=1
        molM=49.988991
    elif iso==3:
        isoN=686
        Sn=0.001991
        Q296=3647.08
        gj=1
        molM=49.988991
    elif iso==4:
        isoN=667
        Sn=7.404750E-04
        Q296=43330.85
        gj=6
        molM=48.98896
    elif iso==5:
        isoN=676
        Sn=3.702370E-04
        Q296=21404.96
        gj=6
        molM=48.98896
    return isoN, Sn, Q296, gj, molM

Функция, считывающая параметры линии поглощения

In [6]:
def readline(file):
    data = scipy.io.loadmat(file) # файл с инфой про линии поглощения для нужного спектрального диапазона
    lines = np.ravel(data['lines'])[0]
    iso = np.ravel(lines[1]) # массив с номерами изотопов
    v = np.ravel(lines[2]) # массив волновых чисел
    S = np.ravel(lines[3]) # массив интенсивностей линий
    A = np.ravel(lines[4]) # массив коэффициентов Эйнштейна
    air = np.ravel(lines[5]) # массив HMHM(air)
    self = np.ravel(lines[6]) # массив HWHM(self)
    E = np.ravel(lines[7]) # массив энергий нижних состояний
    nt = np.ravel(lines[8]) # массив коэффициентов зависимостей air от T
    dlt = np.ravel(lines[9]) # массив сдвигов линий
    return iso, v, S, A, air, self, E, nt, dlt

Функция для подсчёта сечений

In [11]:
def cross(T, p, w1, w2, isn):
    # константы
    R = 8.31441 # универсальная газовая постоянная [Дж/(мол*К)]
    c = 299792458. # скорость света [м/с]
    c2 = 1.438775221 # вторая постоянная Планка [см*к] (c2=hc/kB)
    
    iso, v, S, A, air, self, E, nt, dlt = readline('03_hit16.mat')
    wstep = 0.001 # см^-1
    
    if p >= 1:
        wLim0 = 30. # расширение диапазона
    elif (p < 1)&(p >= 0.1):
        wLim0 = 25.
    elif (p < 0.1)&(p >= 0.01):
        wLim0 = 20.
    elif (p < 0.01)&(p >= 1e-04):
        wLim0 = 15.
    elif (p < 1e-04)&(p >= 1e-07):
        wLim0 = 10.
    elif p < 1e-07:
        wLim0 = 5.
            
    if isn == 100: # номер изотопа (100 - все изотопы)
    # фильтрация, оставляем только нужные нам волновые числа
        fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0))
        numiso = int(np.max(iso)) # кол-во изотопов
        QQT = np.zeros(numiso) # это Q(T) для каждого изотопа
        Sn=np.zeros(numiso) # это распространенность изотопа в атмосфере
        Q296=np.zeros(numiso) # это Q(296 К) для каждого изотопа
        mu=np.zeros(numiso) # это молярная масса каждого изотопа [г/моль]
        for i in range(numiso): 
            isoN1, Sn1, Q296_1, gj1, mu1 = read_molpar16_O3(i + 1)
            Sn[i]=Sn1
            Q296[i]=Q296_1
            mu[i]=mu1
        QQT = do_Qt16_O3(T)
    else:
        fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (iso == isn))
        QQT0_arr = do_Qt16_O3(T)
        QQT0 = QQT0_arr[isn - 1]
        isoN0, Sn0, Q0, gj0, mu0 = read_molpar16_O3(isn)
            
    iso = iso[fv]
    v = v[fv]
    S = S[fv]
    A = A[fv]
    air = air[fv]
    self = self[fv]
    E = E[fv]
    nt = nt[fv]
    dlt = dlt[fv]
    nW = len(v) # кол-во волновых чисел
        
    # сетка
    wGas = np.arange(w1 - wLim0, w2 + wLim0, wstep)
    nG = len(wGas)  # размер сетки
    # если массив волновых чисел в итоге пуст, то cGas = ...
    if v.size == 0:
        cGas = (1.0e-150)*np.ones(nG)
        
    # сортировка массивов по возрастанию v
    ix = np.argsort(v)
    v = np.take(v, ix)
    iso = np.take(iso, ix)
    S = np.take(S, ix)
    A = np.take(A, ix)
    air = np.take(air, ix)
    self = np.take(self, ix)
    E = np.take(E, ix)
    nt = np.take(nt, ix)
    dlt = np.take(dlt, ix)
    # убираем повторение v(j)
    if nW > 1:
        # прогоняем один раз
        dv = v[1:len(v)] - v[0:len(v) - 1] # массив
        for i in range(len(dv)):
            if dv[i] < 5e-07:
                dv[i] = 5e-07
        for j in range(1, len(v)):
                v[j] = v[j - 1] + dv[j - 1]
        # прогоняем второй раз
        dv = v[1:len(v)] - v[0:len(v) - 1]
        for i in range(len(dv)):
            if dv[i] < 5e-08:
                dv[i] = 5e-08
        for j in range(1, len(v)):
            v[j] = v[j - 1] + dv[j - 1]
    
    wL = np.empty(nW)
    for g in range(nW):
        wL[g] = 1.5*air[g]*(p/1013.)*((296./T)**nt[g]) # уширение за счет столкновений [см^-1]
    cGas = (1.0e-150)*np.ones(nG)
    S0 = np.zeros(nW)
    for g in range(nW):
        if isn == 100:
            dQ = Q296[int(iso[g])-1]/QQT[int(iso[g])-1] # отношение Q296/Q(T)
            wD = np.sqrt(2*1000*R*T/mu[int(iso[g])-1])*v[g]/c # Доплеровское уширение
            Rn=1.0
        else:
            dQ = Q0/QQT0 # отношение Q296/Q(T)
            wD = np.sqrt(2*1000*R*T/mu0)*v[g]/c
            Rn = 1/Sn0 # нормировка
            
        # подсчёт S(T)
        St = Rn*S[g]*dQ*np.exp(c2*E[g]*((1/296.) - (1/T)))*(1 - np.exp(-c2*v[g]/T))/(1 - np.exp(-c2*v[g]/296.))
        wLim = 10.
        wFind = np.where((wGas >= v[g] - wLim) & (wGas <= v[g] + wLim))
        wLine = wGas[wFind]

        fV = (1.0e-150)*np.ones(nG)
        fV0 = contur_Humlicek(wLine, v[g], wL[g], wD)
        fV[wFind] = St*fV0
        cGas = cGas + fV
        
    cGas = cGas[np.where((wGas >= w1) & (wGas <= w2))]
    wGas = wGas[np.where((wGas >= w1) & (wGas <= w2))]
    return cGas, wGas

In [8]:
# профили T, p 
data1 = scipy.io.loadmat('pos12_SEC_Tp_co2_0-100km_3155-3220cm-1.mat')
T = np.ravel(data1['temp'])
P = np.ravel(data1['pres'])
alt = np.ravel(data1['alt'])

In [9]:
data_custom = dict.fromkeys(['SECmat_o3', 'alt', 'pres', 'temp', 'wGas'])
data_custom['alt'] = alt
data_custom['pres'] = P
data_custom['temp'] = T

In [12]:
for i in range(len(P)):
    cGas_, wGas = cross(T[i], P[i], 2900, 3200, 100)
    if (i == 0):
        cGas = np.empty(len(cGas_))
    # создание матрицы сечений
    cGas = np.row_stack((cGas, cGas_))
    print(i, cGas_)
data_custom['wGas'] = wGas
data_custom['SECmat_o3'] = cGas
scipy.io.savemat('secmat_o3.mat', data_custom)

0 [  8.23470501e-31   7.55723963e-31   7.05397110e-31 ...,   1.82566402e-28
   4.67246000e-28   2.02794113e-27]
1 [  1.48318958e-30   1.35924330e-30   1.26731039e-30 ...,   3.05731935e-28
   6.63470172e-28   2.55475432e-27]
2 [  2.90746911e-30   2.66223927e-30   2.48052161e-30 ...,   5.56733162e-28
   9.79838977e-28   3.13177516e-27]
3 [  5.84795052e-30   5.34104752e-30   4.96733376e-30 ...,   1.06501861e-27
   1.75054954e-27   5.00752545e-27]
4 [  1.08434519e-29   9.88257269e-30   9.18006757e-30 ...,   1.88494149e-27
   3.08983462e-27   8.43033364e-27]
5 [  2.02130431e-29   1.84118348e-29   1.71019230e-29 ...,   3.32512049e-27
   4.98790960e-27   1.19561925e-26]
6 [  3.80630768e-29   3.46682003e-29   3.22066537e-29 ...,   5.97447144e-27
   8.02368114e-27   1.61053145e-26]
7 [  6.61196512e-29   6.02229851e-29   5.59523969e-29 ...,   1.00930709e-26
   1.24809253e-26   2.12513170e-26]
8 [  1.15633025e-28   1.05322300e-28   9.78549419e-29 ...,   1.73837008e-26
   2.01908273e-26   2.947689